# Laden der Daten
In diesem Notebook werden die Standorte der schweizer Handyantennen aus der Datenquelle data.geo.admin.ch eingelesen. Überflüssige spalten werden entfernt, koordinaten werden ins WSG84 format umgewandelt. Und die Adressdaten werden per Nominatim abgerufen, verarbeitet und im Dataframe erweitert

Autor: Niklaus Haenggi
E-Mail: haengnik@students.zhaw.ch
Created: 28.4.2023


## Verwendete Links/Quellen

Letzter Zugriff: 01.12.2021

In [1]:
url_5g="http://data.geo.admin.ch/ch.bakom.mobil-antennenstandorte-5g/data/ch.bakom.mobil-antennenstandorte-5g_de.json"
url_lte='http://data.geo.admin.ch/ch.bakom.mobil-antennenstandorte-lte/data/ch.bakom.mobil-antennenstandorte-lte_de.json'
url_umts='http://data.geo.admin.ch/ch.bakom.mobil-antennenstandorte-umts/data/ch.bakom.mobil-antennenstandorte-umts_de.json'
url_gsm='http://data.geo.admin.ch/ch.bakom.mobil-antennenstandorte-gsm/data/ch.bakom.mobil-antennenstandorte-gsm_de.json'
url_plz = 'https://swisspost.opendatasoft.com/explore/dataset/plz_verzeichnis_v2/table/?disjunctive.postleitzahl&geofilter.distance='
url_stops = 'https://data.geo.admin.ch/ch.bav.haltestellen-oev/haltestellen-oev/haltestellen-oev_2056_de.csv.zip'


***

## Installation notwendiger Bibliotheken

In [2]:
!pip install --upgrade pip
!pip install folium
!pip install matplotlib
!pip install geopandas
!pip install geopy
!pip install pyproj
!pip install certifi
!pip install ssl


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple
     ---------------------------------------- 2.1/2.1 MB 13.1 MB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\Anaconda3\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror

  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [25 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 14, in <module>
    File "C:\Anaconda3\lib\site-packages\setuptools\__init__.py", line 189, in <module>
      monkey.patch_all()
    File "C:\Anaconda3\lib\site-packages\setuptools\monkey.py", line 99, in patch_all
      patch_for_msvc_specialized_compiler()
    File "C:\Anaconda3\lib\site-packages\setuptools\monkey.py", line 169, in patch_for_msvc_specialized_compiler
      patch_func(*msvc14('_get_vc_env'))
    File "C:\Anaconda3\lib\site-packages\setuptools\monkey.py", line 149, in patch_params
      mod = import_module(mod_name)
    File "C:\Anaconda3\lib\importlib\__init__.py", line 127, in import_module
      return _bootstrap._gcd_import(name[level:], package, level)
    File "C:\Anaconda3\lib\site-packages\setuptools\_distutils\

***

## Module importieren

In [3]:
import csv

# library to handle data in a vectorized manner 
import numpy as np 

# library for data analysis
import pandas as pd

import geopandas as gpd

# library to handle JSON files
import json

# transform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

# library to show coordinates on a map
import folium
from folium import plugins

# library to handle requests
import requests

# to extract csv from haltestellen-oev_2056_de.csv.zip
import zipfile

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# necessary for loading url data
import io

from geopy.geocoders import Nominatim

import ssl
import certifi

import geopy.geocoders

import time


from pyproj import Proj, transform

print('Libraries imported.')



from geopy.exc import GeocoderTimedOut



Libraries imported.


***

# Functions

### function convert_lv95_wgs8
this function is here to recalculate coordinates from the global lv95 format to the swiss wgs84 format

In [4]:
# https://www.swisstopo.admin.ch/content/swisstopo-internet/de/online/calculation-services/_jcr_content/contentPar/tabs/items/dokumente_und_publik/tabPar/downloadlist/downloadItems/8_1467103085694.download/refsys_d.pdf)
# Zugriff: 08.04.2023
def convert_lv95_wgs84(e,n): #Funktion Umrechnung lv95 zu wgs84
    
    y = (e - 2600000)/1000000
    x = (n - 1200000)/1000000
    
    l = 2.6779094 + 4.728982 * y + 0.791484 * y * x + 0.1306 * y * x**2 - 0.0436 * y**3
    b = 16.9023892 + 3.238272 * x - 0.270978 * y**2 - 0.002528 * x**2 - 0.0447 * y**2 * x - 0.0140 * x**3
    
    laenge_wgs84 = l * 100 / 36
    breite_wgs84 = b * 100 / 36
    
    return breite_wgs84, laenge_wgs84



### function load_data

This class load_data was implemented to be able to read anttennen - zip code data with one command.
This does not work for stops, because they are not formatted according to UTF-8.

Die Klasse load_data wurde implementiert, um  Anttennen - Postleitzahlendaten mit einem Befehl einlesen zu können. <br>
Diese funktioniert nicht für Haltestellen, da diese nicht nach UTF-8 formatiert sind


In [5]:
class load_data:
    def __init__(self,path):
        self.path=path

    @staticmethod
    def load_from_url(url):
        s=requests.get(url).content
        df=pd.read_csv(io.StringIO(s.decode('utf-8')))
        return df

        
    @staticmethod
    def load_pkl(path_to_pkl):
        with open(path_to_pkl,'rb') as f:
            data=pickle.load(f)
        return data
        
    @staticmethod
    def load_csv(path_to_csv):
        df=pd.read_csv(path_to_csv,sep=',')
        return df

***

### function convert_panda_lv95_wgs84

this function converts a pandas dataframe which contains coordinates from the global lv95 format to the swiss wgs84 format.


In [6]:
# LV95 zu WGS84
def convert_panda_lv95_wgs84(dataFrame ): #Funktion Umrechnung lv95 zu wgs84
    inProj = Proj('epsg:2056')
    outProj = Proj('epsg:4326')
    t = dataFrame['cordinates'].x
    x1,y1 = dataFrame['cordinates'].x, dataFrame['cordinates'].y
    x2,y2 = transform(inProj, outProj, x1,y1)
    dataFrame['antenna_Location_WGS84_E'] = x2
    dataFrame['antenna_Location_WGS84_N'] = y2
    return dataFrame

### function get_address
this function is here to catch the address with coordinates on openstreetmap by using nominatim
it converts the address string to a list

In [7]:
def get_address(coordinates, attempt=1, max_attempts=7):
    try:
        locator = Nominatim(user_agent="add_location_adress")
        location = locator.reverse(coordinates)
        address = location.address.split(",")
        countries = address[len(address)-1].split("/")
        address[len(address)-1] = countries[0]
        return address
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            return get_kreis(coordinates, attempt=attempt+1)
        raise

### function add_address
because nominatim returns the address not always in the same format, we can use this function to get tha address in correct order

for example the return of nominatim can be:
['building', 'streetNumber' , 'street' , 'town' , 'district' , 'plz' , 'state']
or
['street' , 'town' , 'district' , 'plz' , 'state']
so the street is not always on the 3rd place

this function will sort that in the correct order and put 'None' values if something is missing

In [8]:
def add_address(data_frame ):

# set all address fields to None initial    
    data_frame['building'] = None
    data_frame['streetNumber'] = None
    data_frame['street'] = None
    data_frame['town'] = None
    data_frame['district'] = None
    data_frame['state'] = None

# set a counter to stop nominatim requests after a speficic number of requests
    counter = 0
    
    
    for index, row in data_frame.iterrows():
        
        
# set counters for adress length forward an backward          
        forward = 0
        backward = len(adresse)-1
        
        
        
# read coordinates of antenna        
        coordinates = [row['antenna_Location_WGS84_E'],row["antenna_Location_WGS84_N"]]

# get address of antenna by nominatim
        adresse = get_address(coordinates)

# set counters for address length forward and backward        
        forward = 0
        backward = len(adresse)-1
    
# check if first or second entry is street number and update address data of building and streetnumber
        if adresse[forward][0].isdigit() or len(adresse[forward]) >1 and adresse[forward][1].isdigit():
            data_frame.loc[index , "streetNumber"] = adresse[0]
            street_number.append(adresse[0])
            forward += 1
        elif adresse[forward+1][0].isdigit() or len(adresse[forward+1]) >1 and adresse[forward+1][1].isdigit():
            data_frame.loc[index , "streetNumber"] = adresse[forward+1]
            data_frame.loc[index , "building"] = adresse[forward]
            forward += 2

        else:
            street_number.append(None)


# update street with next field of address list
        street.append(adresse[forward])
        data_frame.loc[index , "street"] = adresse[forward]



# add country from the end of the address list
        country.append(adresse[backward])
        data_frame.loc[index , "country"] = adresse[backward]

        backward -= 1




# check in which position of the address list is the postal code and update postal code in dataframe
        if len(adresse[backward]) == 5 and adresse[backward][1].isdigit():
            data_frame.loc[index , "postal"] = adresse[backward]
            data_frame.loc[index , "town"] = plz.get(int(adresse[backward]))
        elif len(adresse[backward+1]) == 5 and adresse[backward+1][1].isdigit():
            data_frame.loc[index , "postal"] = adresse[backward+1]
            data_frame.loc[index , "town"] = plz.get(int(adresse[backward+1]))
            backward = backward + 1
        elif len(adresse[backward-1]) == 5 and adresse[backward-1][1].isdigit():
            data_frame.loc[index , "postal"] = adresse[backward-1]
            data_frame.loc[index , "town"] = plz.get(int(adresse[backward-1]))
            backward = backward - 1





        print(str(backward) + "   " + adresse[backward])






# go 3 steps frim town in the address list
        backward -= 3
    
    
        print(str(backward) + "    " + adresse[backward])


# add state to dataframe
        data_frame.loc[index , "state"] = adresse[backward]
        backward -= 1

# check if district is in the address
        if adresse[backward].__contains__("ezirk") or adresse[backward].__contains__("istrict") or adresse[backward].__contains__("istretto") or adresse[backward].__contains__("erwaltungsregion") or adresse[backward].__contains__("égion administrative") or adresse[backward].__contains__("ahlkreis") or adresse[backward].__contains__("Region"):
            district.append(adresse[backward])
            data_frame.loc[index , "district"] = adresse[backward]
            backward -= 1
        else:
            district.append(None)

        town.append(adresse[backward])
        backward -= 1




# set pause after specific number of requests to not get blocked by OpenStreetMap because of too many requests

        counter += 1
        if counter % 150 == 0:
            time.sleep(20)

        if counter % 9000 == 0:
            time.sleep(1800)


    return data_frame






## 1) Import mapping dictionary for postal codes

<br>This dataset provides thisSwiss postal codes with the associated place names from which a dictionary is created to look up all place names later <br/>.

<br>Dieser Datensatz liefert dieschweizer Postleitzahlen mit den dazugehöhrigen Ortsnamen dazaus wird ein dictionary erstellt, um später alle Ortsnamen nachzuschlagen <br/>

In [9]:
plz=pd.read_csv('./Daten/plz_verzeichnis_v2.csv',sep=';')

# just keep plz and village of plz dataframe
plz = plz[['POSTLEITZAHL', 'ORTBEZ18']]

# rename columns
plz = plz.rename({'POSTLEITZAHL': 'PLZ', 'ORTBEZ18': 'ORT'}, axis=1)  # new method

#create dictionary
plz = plz.set_index('PLZ').to_dict()['ORT']



In [10]:
## short unreal dataset of antennas for testing reasons

In [11]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-6g_de.json'
antennen_6g = gpd.read_file(data_path)
antennen_6g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Mittel,POINT (2688674.000 1211488.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Sehr Klein,POINT (2666065.000 1214355.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Mittel,POINT (2722651.000 1271244.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2570323.000 1164067.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Klein,POINT (2613427.000 1262637.000)


## 2) Import locations of antena
<br> JSON from antenna to Pandas <br>

In [12]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-5g_de.json'
antennen_5g = gpd.read_file(data_path)
antennen_5g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Mittel,POINT (2688674.000 1211488.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Sehr Klein,POINT (2666065.000 1214355.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Mittel,POINT (2722651.000 1271244.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2570323.000 1164067.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Klein,POINT (2613427.000 1262637.000)


In [13]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-lte_de.json'
antennen_4g = gpd.read_file(data_path)
antennen_4g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Gross,POINT (2558525.000 1167925.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Gross,POINT (2648322.000 1246675.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Gross,POINT (2503428.000 1117642.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2601088.000 1200806.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Gross,POINT (2567350.000 1228990.000)


In [14]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-umts_de.json'
antennen_3g = gpd.read_file(data_path)
antennen_3g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Mittel,POINT (2601088.000 1200806.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Gross,POINT (2639748.000 1232218.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Mittel,POINT (2613427.000 1262637.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2503428.000 1117642.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Klein,POINT (2570323.000 1164067.000)


In [15]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-gsm_de.json'
antennen_2g = gpd.read_file(data_path)
antennen_2g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Mittel,POINT (2701702.000 1265811.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Mittel,POINT (2618398.000 1227312.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Mittel,POINT (2609644.000 1213344.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2560331.000 1182192.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Sehr Klein,POINT (2557146.000 1210825.000)


## 3) Import locations of public transportation stops
<br> CSV of stops to Pandas <br>

In [16]:

stops = pd.read_csv('./Daten/Haltekante.csv', sep=',', encoding='ISO-8859-1')
stops.head(200)


,xtf_id,Nummer,Bezeichnung,BetrieblicheBezeichnung,Laenge,Kantenhoehe,Gueltigkeit_BeginnGueltigkeit,Gueltigkeit_EndeGueltigkeit,Gueltigkeit_Stand,rHaltestelle,E,N,H
0,ch14uvag00259759,ch:1:sloid:79981:0:97116,NaN,WABERÜT2,NaN,NaN,20211212,NaN,20230327,ch14uvag00063208,2741304.000,1260064.000,602.0
1,ch14uvag00264480,ch:1:sloid:70927:0:28883,NaN,ERBIOZ2,NaN,NaN,20211212,NaN,20230327,ch14uvag00054412,2598366.000,1118588.000,823.0
2,ch14uvag00239449,ch:1:sloid:30581:0:739922,NaN,1,NaN,NaN,20210211,NaN,20230327,ch14uvag00038841,2758941.000,1183440.001,1725.1
3,ch14uvag00237611,ch:1:sloid:31297:0:898605,NaN,1,NaN,NaN,20210211,NaN,20230327,ch14uvag00220548,2820554.000,1206750.000,2745.9
4,ch14uvag00247605,ch:1:sloid:77603:0:2,NaN,2,15.0,0.0,20220301,NaN,20230327,ch14uvag00058363,2690364.000,1215382.000,784.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,ch14uvag00247067,ch:1:sloid:77132:0:1,NaN,1,NaN,NaN,20220301,NaN,20230327,ch14uvag00055010,2669768.000,1228001.000,528.0
196,ch14uvag00234554,ch:1:sloid:78954:0:2,NaN,2,NaN,NaN,20200703,NaN,20230327,ch14uvag00054315,2608794.417,1192083.631,526.6
197,ch14uvag00241425,ch:1:sloid:87898:0:01,NaN,1,NaN,NaN,20210401,NaN,20230327,ch14uvag00041045,2589014.000,1223463.000,437.0
198,ch14uvag00247685,ch:1:sloid:93393:0:2,NaN,2,NaN,NaN,20220301,NaN,20230327,ch14uvag00048737,2663673.000,1223911.000,475.0


In [17]:
stops=pd.read_csv('./Daten/Betriebspunkt.csv',sep=',' , encoding = 'ISO-8859-1')

stops = stops[(stops['Betriebspunkttyp_Bezeichnung'] == 'Haltestelle') | (stops['Betriebspunkttyp_Bezeichnung'] == 'Haltestelle und Bedienpunkt')]

stops.head(200)

,xtf_id,Name,Nummer,Abkuerzung,Transportunternehmen_Nummer,Transportunternehmen_Abkuerzung,Betriebspunkttyp_Code,Betriebspunkttyp_Bezeichnung,Verkehrsmittel_Code,Verkehrsmittel_Bezeichnung,Gemeinde_Nummer,Gemeinde_Name,Gueltigkeit_BeginnGueltigkeit,Gueltigkeit_EndeGueltigkeit,Gueltigkeit_Stand,rUebergeordneteHaltestelle,E,N,H
0,ch14uvag00047721,"Griesenberg, Hub",8506971,NaN,7,PAG,VP,Haltestelle,A,Bus,4881,Amlikon-Bissegg,19930201,NaN,20230424,NaN,2719715,1268770,559
1,ch14uvag00051854,"Leukerbad, Therme/Rehazentrum",8594512,NaN,141,LLB,VP,Haltestelle,A,Bus,6111,Leukerbad,20110321,NaN,20230424,NaN,2614423,1136296,1374
2,ch14uvag00052870,"Martigny, Fond. Gianadda",8570961,NaN,325,BUM,VP,Haltestelle,A,Bus,6136,Martigny,19930201,NaN,20230424,NaN,2571423,1105128,470
3,ch14uvag00056951,Rhäzüns,8509185,RHAE,154,RhB FR VR,VPG,Haltestelle und Bedienpunkt,B,Zug,3723,Rhäzüns,19871231,NaN,20230424,NaN,2749660,1185038,654
5,ch14uvag00044446,"Dietwil, Wannenmatt",8577141,NaN,185,ZVB,VP,Haltestelle,A,Bus,4231,Dietwil,19930201,NaN,20230424,NaN,2672576,1221568,408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,ch14uvag00044616,"Dörflingen, Unterberg",8573746,NaN,539,VBSH,VP,Haltestelle,A,Bus,2915,Dörflingen,19930201,NaN,20230424,NaN,2695765,1284695,446
206,ch14uvag00039335,"Arosio, Bassa",8575363,NaN,7,PAG,VP,Haltestelle,A,Bus,5237,Alto Malcantone,19930201,NaN,20230424,NaN,2713247,1100507,829
207,ch14uvag00049126,"Igis, Castaletweg",8581040,NaN,7,PAG,VP,Haltestelle,A,Bus,3955,Landquart,19960602,NaN,20230424,NaN,2762110,1201899,533
208,ch14uvag00279580,"Cadro, Scuole",8510742,NaN,145,ARL,VP,Haltestelle,A,Bus,5192,Lugano,20221211,NaN,20230424,NaN,2719522,1100340,440


# Prepare Datasets

## 1) public transporation stops

In [18]:
### remove unnecessary information

In [19]:
stops.drop(["xtf_id","Nummer","Abkuerzung", "Transportunternehmen_Nummer","Betriebspunkttyp_Code","Verkehrsmittel_Code","Gemeinde_Nummer","Gueltigkeit_BeginnGueltigkeit","Gueltigkeit_EndeGueltigkeit","Gueltigkeit_Stand","rUebergeordneteHaltestelle"], axis=1, inplace=True)

stops.head(200)

,Name,Transportunternehmen_Abkuerzung,Betriebspunkttyp_Bezeichnung,Verkehrsmittel_Bezeichnung,Gemeinde_Name,E,N,H
0,"Griesenberg, Hub",PAG,Haltestelle,Bus,Amlikon-Bissegg,2719715,1268770,559
1,"Leukerbad, Therme/Rehazentrum",LLB,Haltestelle,Bus,Leukerbad,2614423,1136296,1374
2,"Martigny, Fond. Gianadda",BUM,Haltestelle,Bus,Martigny,2571423,1105128,470
3,Rhäzüns,RhB FR VR,Haltestelle und Bedienpunkt,Zug,Rhäzüns,2749660,1185038,654
5,"Dietwil, Wannenmatt",ZVB,Haltestelle,Bus,Dietwil,2672576,1221568,408
...,...,...,...,...,...,...,...,...
205,"Dörflingen, Unterberg",VBSH,Haltestelle,Bus,Dörflingen,2695765,1284695,446
206,"Arosio, Bassa",PAG,Haltestelle,Bus,Alto Malcantone,2713247,1100507,829
207,"Igis, Castaletweg",PAG,Haltestelle,Bus,Landquart,2762110,1201899,533
208,"Cadro, Scuole",ARL,Haltestelle,Bus,Lugano,2719522,1100340,440


### transform cordinates of stops to WSG84

In [20]:
# switch key and value in plz dictionary and save it to a town dictionary
# this will be switched because search for a key in a dictionary is much more efficient than search for the value
towns = {y: x for x, y in plz.items()}

inProj = Proj('epsg:2056')
outProj = Proj('epsg:4326')

x1,y1 = stops['E'], stops['N']
x2,y2 = transform(inProj, outProj, x1,y1)
stops['stop_location_WGS84_E'] = x2
stops['stop_location_WGS84_N'] = y2


stops.drop(["E","N"])
# after we have 'WGS84' - coordinates, we don't need them anymore



C:\Users\i00202914\AppData\Local\Temp\ipykernel_21120\3278909290.py:9: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2,y2 = transform(inProj, outProj, x1,y1)


KeyError: "['E', 'N'] not found in axis"

### add Postal code to stops
try to find out the postal according the village name in the town dictionary


In [ ]:
# try to find postal code according village name
for index, row in stops.iterrows():
    stops.loc[index , "postal"] = str(towns.get(row["Gemeinde_Name"]))

stops.head(200)

In [ ]:
# remove unused columns from stops dataframe

In [ ]:
stops.drop(["xtf_id","Bezeichnung","BetrieblicheBezeichnung","Laenge","Kantenhoehe","Gueltigkeit_BeginnGueltigkeit","Gueltigkeit_EndeGueltigkeit","Gueltigkeit_Stand","rHaltestelle"], axis=1, inplace=True)
columns=['number', 'cordinate_e', 'cordinate_n', 'high']
columns=['number', 'antenna_Location_WGS84_E', 'antenna_Location_WGS84_N', 'high']

# stops.columns = columns
# stops.head()



# stops = add_address(stops)
stops.head()



## 2) Antennas



### clean columns
<br> Set column names and drop usless Columns <br>

In [ ]:

columns=['language', 'description', 'id', 'power', 'cordinates']
antennen_6g.columns=columns
antennen_5g.columns=columns
antennen_4g.columns=columns
antennen_3g.columns=columns
antennen_2g.columns=columns

In [ ]:
antennen_6g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_5g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_4g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_3g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_2g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_4g.head(4)

### get address data from openstreet map

In [ ]:
### create self-signed certificate for doing requests with nominatim

In [ ]:
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
print(certifi.where())

### convert antenna coordinates of a dataframe from LV95 to WGS84

convert coordinates of antennas from lv95 to wgs84 with previous defined function

In [ ]:
antennen_6g = convert_panda_lv95_wgs84(antennen_6g)
antennen_5g = convert_panda_lv95_wgs84(antennen_5g)
antennen_4g = convert_panda_lv95_wgs84(antennen_4g)
antennen_3g = convert_panda_lv95_wgs84(antennen_3g)
antennen_2g = convert_panda_lv95_wgs84(antennen_2g)



### add address data to data frames for display them later on the map
this task adds the address data to the data frame with the previous defined function using nominatim
it will take a lot of time

In [ ]:
antennen_6g = add_address(antennen_6g)
# antennen_5g = add_address(antennen_5g)
# antennen_4g = add_address(antennen_4g)

# antennen_3g = add_address(antennen_3g)
# antennen_2g = add_address(antennen_2g)


### add type of antenna to antenna data frames 

In [ ]:
antennen_6g = antennen_6g.assign(type='6G')
antennen_5g = antennen_5g.assign(type='5G')
antennen_4g = antennen_4g.assign(type='4G')
antennen_3g = antennen_3g.assign(type='3G')
antennen_2g = antennen_2g.assign(type='2G')

antennen_6g.head()


##  remove special chars for antenna and stop dataframes
not all systems are supporting special chars, so they should be better replaced

In [ ]:

# Rename special chars
def rename_Special(data_frame):
    data_frame = data_frame.replace({'dictionary':{'ä':'ae','Ä':'Ae','Ü':'Ue','ü':'ue','Ö':'Oe','ö':'oe'}},regex=True)
    return data_frame



In [ ]:
antennen_6g = rename_Special(antennen_6g)
antennen_5g = rename_Special(antennen_5g)
antennen_4g = rename_Special(antennen_4g)
antennen_3g = rename_Special(antennen_3g)
antennen_2g = rename_Special(antennen_2g)
stops = rename_Special(stops)


# Save dataframes in CSV
after all dataframes are ready to use, they will be saved in a CSV file.
Nominatim takes a long time to get all information of all datasets, so it is important to save the frames after the adress is added.

In [ ]:
antennen_6g.to_csv('./Daten/6gAntennen.csv',sep=';', index=False)
antennen_5g.to_csv('./Daten/5gAntennen.csv',sep=';', index=False)
antennen_4g.to_csv('./Daten/4gAntennen.csv',sep=';', index=False)
antennen_3g.to_csv('./Daten/3gAntennen.csv',sep=';', index=False)
antennen_2g.to_csv('./Daten/2gAntennen.csv',sep=';', index=False)
stops.to_csv('./Daten/stops.csv',sep=';', index=False)

# Load saved dataframes from CSV
to skip previous steps, we can continue with the prevous saved dataframes, wich conatains already the adresses

In [ ]:
antennen_5g=pd.read_csv('./Daten/5gAntennen.csv',sep=';')
antennen_4g=pd.read_csv('./Daten/4gAntennen.csv',sep=';')
antennen_3g=pd.read_csv('./Daten/3gAntennen.csv',sep=';')
antennen_2g=pd.read_csv('./Daten/2gAntennen.csv',sep=';')
stops=pd.read_csv('./Daten/stops.csv',sep=';')




antennen_5g.head()

In [ ]:
# Prepare map
preparing a folium maps which is showing switzerland

In [ ]:
print(certifi.where())
address = 'Schweiz'

geolocator = Nominatim(user_agent="antenna")
location = geolocator.geocode(address)

latitude = location.latitude
longitude = location.longitude

In [ ]:
latitude = 46.9990
longitude =8.0008

Erster Plot zum anschauen des Netzes:

In [ ]:
# workaround for SSL Error:
import ssl
import certifi
import geopy.geocoders
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
print(certifi.where())

In [ ]:
### Add antenas to the map

In [ ]:
m=folium.Map(location=[latitude,longitude],zoom_start=9)

# function add_to_map
this function is setting the marks on the map for each antenna
the mark conatains the adress and type of the antenna is set by the color
GSM = blue
UMTS = green
LTE = Orange
5G = RED


In [ ]:
map_stops=folium.Map(location=[latitude,longitude],zoom_start=9)
stops.head()

In [ ]:


def add_antenna_to_map(dataframe):
    color = 'white'

    if dataframe['type'][0] == '2G':
        color = 'blue'
    if dataframe['type'][0] == '3G':
        color = 'green'
    if dataframe['type'][0] == '4G':
        color = 'orange'
    if dataframe['type'][0] == '5G':
        color = 'red'


    print(dataframe['type'][0])

    #You Markler the point in Map
    for indice, row in dataframe.iterrows():

        color = 'white'

        if row['type'][0] == '2G':
            color = 'blue'
        if row['type'][0] == '3G':
            color = 'green'
        if row['type'][0] == '4G':
            color = 'orange'
        if row['type'][0] == '5G':
            color = 'red'

     #   print(row['building'])
        folium.Marker(
            location=[row['antenna_Location_WGS84_E'], row['antenna_Location_WGS84_N']],
            # popup=row,

            popup=[row['building'] , row["streetNumber"] ,row['postal'],  row['town'] , row['district'] , row['state'] ],
            icon=folium.map.Icon(color=color)
        ).add_to(m)



# function add_antenna_to_map
this function is setting the marks on the map for each stop
    the mark conatains the adress and type of the stop is set by the color
T
rain = darkblue
Ship = green
Bus = orange
Tram = brown
Metro = red
Gear train = grey
Cabin =  brown
Elevator =

In [ ]:

def add_stops_to_map(dataframe):
    color = 'white'



    if dataframe['Verkehrsmittel_Bezeichnung'].__contains__('Zug'):
        color = 'darkblue'
    elif dataframe['Verkehrsmittel_Bezeichnung'].__contains__('Schiff'):
        color = 'green'
    elif dataframe['Verkehrsmittel_Bezeichnung'].__contains__('Bus'):
        color = 'orange'
    elif dataframe['Verkehrsmittel_Bezeichnung'].__contains__('Tram'):
        color = 'brown'
    elif dataframe['Verkehrsmittel_Bezeichnung'].__contains__('Metro'):
            color = 'red'
    elif dataframe['Verkehrsmittel_Bezeichnung'].__contains__('Zahnradbahn'):
        color = 'grey'
    elif dataframe['Verkehrsmittel_Bezeichnung'].__contains__('Kabinenbahn'):
        color = 'brown'
    elif dataframe['Verkehrsmittel_Bezeichnung'].__contains__('Aufzug'):
        color = 'blue'
    print(dataframe['Verkehrsmittel_Bezeichnung'][0])


   # UserWarning: color argument of Icon should be one of: {'white', 'gray', 'lightgray', 'lightgreen', 'blue', 'green', 'beige', 'darkblue', 'black', 'lightred', 'pink', 'darkpurple', 'darkred', 'lightblue', 'cadetblue', 'darkgreen', 'purple', 'orange', 'red'}.

    #You Markler the point in Map
    for indice, row in dataframe.iterrows():
        if row['Verkehrsmittel_Bezeichnung'].__contains__('Zug'):
            color = 'cadetblue'
        elif row['Verkehrsmittel_Bezeichnung'].__contains__('Schiff'):
                color = 'lightgreen'
        elif row['Verkehrsmittel_Bezeichnung'].__contains__('Bus'):
                color = 'darkpurple'
        elif row['Verkehrsmittel_Bezeichnung'].__contains__('Tram'):
                color = 'darkgreen'
        elif row['Verkehrsmittel_Bezeichnung'].__contains__('Metro'):
                color = 'darkred'
        elif row['Verkehrsmittel_Bezeichnung'].__contains__('Zahnradbahn'):
                color = 'gray'
        elif row['Verkehrsmittel_Bezeichnung'].__contains__('Kabinenbahn'):
                color = 'lightgray'
        elif row['Verkehrsmittel_Bezeichnung'].__contains__('Aufzug'):
                color = 'darkblue'


        #   print(row['building'])
        folium.Marker(
            location=[row['stop_location_WGS84_E'], row['stop_location_WGS84_N']],
            # popup=row,

            popup=[row['Name'] , row["Verkehrsmittel_Bezeichnung"] , row["Transportunternehmen_Abkuerzung"] , row['postal'],  row['Gemeinde_Name']  ],
            icon=folium.map.Icon(color=color)
        ).add_to(map_stops)



In [ ]:
add_stops_to_map(stops)
map_stops

In [ ]:
display(map_stops)

In [ ]:
map_stops

In [ ]:
m=folium.Map(location=[latitude,longitude],zoom_start=9)
add_antenna_to_map(antennen_2g)
add_antenna_to_map(antennen_3g)
add_antenna_to_map(antennen_4g)
add_antenna_to_map(antennen_5g)

m

In [ ]:
folium.Map(location= [ antennen_6g['antenna_Location_WGS84_E'] , antennen_6g['antenna_Location_WGS84_N']])
# folium.Choropleth(gdf_routennetz[gdf_routennetz.velostreifen!='0'],line_color='blue',line_weight=5,line_opacity=0.3,columns=['name'],legend_name='hehe').add_to(m)
# folium.Choropleth(gdf_routennetz[gdf_routennetz.veloweg!=0].geometry,line_color='red',line_weight=5,line_opacity=0.3).add_to(m)

In [ ]:

display(m)

***